<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/node_postprocessor/ColbertRerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colbert Rerank

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


[Colbert](https://github.com/stanford-futuredata/ColBERT): ColBERT is a fast and accurate retrieval model, enabling scalable BERT-based search over large text collections in tens of milliseconds.

This example shows how we use Colbert-V2 model as a reranker.

In [ ]:
!pip install llama-index
!pip install llama-index-core
!pip install --quiet transformers torch
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-openai
!pip install llama-index-postprocessor-colbert-rerank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)

Download Data

In [ ]:
!mkdir -p 'data/data8/'
!wget 'https://raw.githubusercontent.com/aishwarya-balaji/reranking/main/data/data8/data8.txt' -O 'data/data8/data8.txt'

--2024-03-31 03:40:41--  https://raw.githubusercontent.com/aishwarya-balaji/reranking/main/data/data8/data8.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1424 (1.4K) [text/plain]
Saving to: ‘data/data8/data8.txt’

data/data8/data8.tx 100%[===================>]   1.39K  --.-KB/s    in 0s      

2024-03-31 03:40:41 (15.7 MB/s) - ‘data/data8/data8.txt’ saved [1424/1424]



In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
documents = SimpleDirectoryReader("./data/data8/").load_data()


In [ ]:
from llama_index.core.node_parser import SentenceWindowNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
text_splitter = SentenceSplitter()
base_nodes = text_splitter.get_nodes_from_documents(documents)

In [ ]:
sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)

In [ ]:
# # load documents
# documents = SimpleDirectoryReader("./data/data1/").load_data()
# from llama_index.core.node_parser import SentenceSplitter

# parser = SentenceSplitter()
# nodes = parser.get_nodes_from_documents(documents)
# print(nodes)

# # build index
# index = VectorStoreIndex(nodes)

[TextNode(id_='9127bac7-3c4c-4fee-a8c2-4f1212f58061', embedding=None, metadata={'file_path': '/content/data/data1/data1.txt', 'file_name': 'data1.txt', 'file_type': 'text/plain', 'file_size': 1679, 'creation_date': '2024-03-25', 'last_modified_date': '2024-03-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2be70d15-763d-47dd-9a2e-9bb3bf2d1b8b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/data/data1/data1.txt', 'file_name': 'data1.txt', 'file_type': 'text/plain', 'file_size': 1679, 'creation_date': '2024-03-25', 'last_modified_date': '2024-03-25'}, hash='60c4ca96560d33c97f376a4047c5e4ab3502a0cb17dcc175e0d7d7d5baccbffe')}, text='London, the capital city of England and the Unit

#### Retrieve top 10 most relevant nodes, then filter with Colbert Rerank

In [ ]:
# trying to build index information

#index = VectorStoreIndex.from_documents(documents=paragraphs)

In [ ]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=3,#will be equal to how many tools are executed
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[colbert_reranker],
)
response = query_engine.query(
    "Why do people visit the Great Wall of China?",
)

In [ ]:
print("Total nodes in response:", len(response.source_nodes))  # This will print how many nodes are there

Total nodes in response: 3


In [ ]:
# TO DO: figure out how to acess scores for all nodes
for node in response.source_nodes:
    print(node.id_)
    print(node.node.get_content()[:150]) # number of characters - but not the paragraphs
    print("reranking score: ", node.score)
    print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("**********")

801d39bb-a035-4eb3-a658-7cd2bdb174ec
The Great Wall played a crucial role in China's history, serving as a defensive barrier against invasions by nomadic tribes from the north, such as th
reranking score:  0.6774057745933533
retrieval score:  0.8438549427329477
**********
37bd34c3-b5c9-4127-aea7-27e0591ffb99
The Great Wall holds immense historical, cultural, and symbolic significance for China and the world. 
reranking score:  0.6759287714958191
retrieval score:  0.8495571378290605
**********
022c7467-d096-41cf-8adb-e5abd7c07d95
The Great Wall of China is one of the most iconic and extensive architectural marvels in human history. 
reranking score:  0.6697898507118225
retrieval score:  0.846457365232407
**********


In [ ]:
print(response)

People visit the Great Wall of China to experience its historical, cultural, and symbolic significance, as well as to witness one of the most iconic and extensive architectural marvels in human history.
